In [1]:
import urdf2casadi.urdfparser as u2c
import numpy as np

## 1. Load robot from urdf 

1. Create urdfparser-class instance. 
2. Load model to instance, either from file, string or ros parameter server. Examples uses from file. 

In [2]:
ur5 = u2c.URDFparser()
ur5.from_file("/home/lmjohann/urdf2casadi/examples/urdf/ur5.urdf")

/home/lmjohann/urdf2casadi/examples/urdf/ur5.urdf


## 2. Get joint information ? 
Information about the joints of the robot model can be obtained using u2c's get_joint_info(). "root" and "tip" are the inputs, and represent the link names for the root and tip of the kinematic tree one wishes to evaluate. 

In [3]:
root = "base_link"
tip = "wrist_3_link"

joint_list, joint_names, q_max, q_min = ur5.get_joint_info(root, tip)
n_joints = len(joint_list)
print "name of first joint:", joint_names[0], "\n"
print "joint information for first joint:\n", joint_list[0]
print "\n q max:", q_max
print "\n q min:", q_min

name of first joint: shoulder_pan_joint 

joint information for first joint:
axis:
- 0.0
- 0.0
- 1.0
calibration: None
child: shoulder_link
dynamics:
  damping: 0.0
  friction: 0.0
limit:
  effort: 150.0
  lower: -6.28318530718
  upper: 6.28318530718
  velocity: 3.15
mimic: None
name: shoulder_pan_joint
origin:
  rpy:
  - 0.0
  - 0.0
  - 0.0
  xyz:
  - 0.0
  - 0.0
  - 0.089159
parent: base_link
safety_controller: None
type: revolute

 q max: [6.28318530718, 6.28318530718, 3.14159265359, 6.28318530718, 6.28318530718, 6.28318530718]

 q min: [-6.28318530718, -6.28318530718, -3.14159265359, -6.28318530718, -6.28318530718, -6.28318530718]


# 3. Obtain robot dynamics! 
After loading the robot model to the urdfparser-instance, the robot dynamics can be obtained from any given root and tip of the robot. 

## Dynamic parameters:
To obtain the dynamic parameters (M, C, G) of the equation of motion tau = M(q)q_ddot + C(q, q_dot)q_dot + G(q):

In [4]:
M_sym = ur5.get_inertia_matrix_crba(root, tip)
C_sym = ur5.get_coriolis_rnea(root, tip)

gravity = [0, 0, -9.81]
G_sym = ur5.get_gravity_rnea(root, tip, gravity)

M_sym, C_sym, G_sym are CasADi symbolic expressions of the ur5's dynamic parameters from given root to tip. The CasADi expressions are C compiled so they can be numerically evaluated efficiently. Example of numerical evaluation is shown below. 

In [5]:
q = [None]*n_joints
q_dot = [None]*n_joints
for i in range(n_joints):
    #to make sure the inputs are within the robot's limits:
    q[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_dot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    
M_num = M_sym(q)
C_num = C_sym(q, q_dot)
G_num = G_sym(q)
print "Numerical Inertia Matrx for random input: \n", M_num 
print "\nNumerical Coriolis term for random input: \n", C_num 
print "\nNumerical gravity term for random input: \n", G_num

Numerical Inertia Matrx for random input: 

[[0.544933, -0.0181893, 0.0871863, -0.132914, 0.199076, -0.00563352], 
 [-0.0181893, 0.299405, 0.143395, -0.0033509, 0.0147113, 0.0148556], 
 [0.0871863, 0.143395, 0.84282, -0.247647, -0.00721473, -0.00846477], 
 [-0.132914, -0.0033509, -0.247647, 0.248127, 0.00721473, 0.00846477], 
 [0.199076, 0.0147113, -0.00721473, 0.00721473, 0.244034, 0], 
 [-0.00563352, 0.0148556, -0.00846477, 0.00846477, 0, 0.0171365]]

Numerical Coriolis term for random input: 
[8.92962, 5.21485, 3.5309, -3.02294, 4.90214, -0.17699]

Numerical gravity term for random input: 
[1.03672e-15, -3.5587, -13.0609, -0.0112017, 0, 0]


## Inverse Dynamics 

Without accounting for gravitational forces:

In [6]:
tau_sym = ur5.get_inverse_dynamics_rnea(root, tip)

Accounting for gravitational forces:

In [7]:
gravity = [0, 0, -9.81]
tau_g_sym = ur5.get_inverse_dynamics_rnea(root, tip, gravity = gravity)

External forces can also be accounted for:

In [8]:
tau_ext = np.ones(n_joints)
tau_fext_sym = ur5.get_inverse_dynamics_rnea(root, tip, f_ext = tau_ext)

RuntimeError: .../casadi/core/matrix.cpp:1143: Assertion "is_scalar()" failed:
Can only convert 1-by-1 matrices to scalars

In [10]:
q = [None]*n_joints
q_dot = [None]*n_joints
q_ddot = [None]*n_joints
for i in range(n_joints):
    #to make sure the inputs are within the robot's limits:
    q[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_dot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_ddot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2

tau_num = tau_sym(q, q_dot, q_ddot)
tau_g_num = tau_g_sym(q, q_dot, q_ddot)
#tau_fext_num = tau_fext_sym(q, q_dot, q_ddot)
print "Numerical inverse dynamics: \n", tau_num 
print "\nNumerical inverse dynamics w/ gravity: \n", tau_g_num 
#print "\nNumerical inverse dynamics w/ external forces: \n", G_num

Numerical inverse dynamics: 
[14.1199, 8.33948, -1.19301, -0.0712788, -1.65662, 0.0369481]

Numerical inverse dynamics w/ gravity: 
[14.1199, -4.14095, 1.53532, 0.040635, -1.65662, 0.0369481]


## Forward Dynamics

urdf2casadi provides two methods for finding the robot's forward dynamics. The first method combines the Recursive Newton-Euler Algorithm (RNEA) and the Composite Rigid Body Algorithm (CRBA) and solves the equation of motion for the joint accelerations. The second method uses the Articulated Body Algorithm (ABA) for forward dynamics. The method that uses ABA is in most cases the most efficient with regard to numerical evaluation, especially if the number of joints are high. (See timing examples for more information.)

In [14]:
tau = np.zeros(n_joints)
qddot_sym = ur5.get_forward_dynamics_crba(root, tip)

In [15]:
qddot_g_sym = ur5.get_forward_dynamics_aba(root, tip, gravity = gravity)


In [17]:
q = [None]*n_joints
q_dot = [None]*n_joints
for i in range(n_joints):
    #to make sure the inputs are within the robot's limits:
    q[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_dot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2

qddot_num = qddot_sym(q, q_dot, tau)
qddot_g_num = qddot_g_sym(q, q_dot, tau)

print "Numerical inverse dynamics: \n", qddot_num 
print "\nNumerical inverse dynamics w/ gravity: \n", qddot_g_num 